In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
# Add massimal tools folder to path
import sys
sys.path.append("/massimal/python/tools")

In [3]:
import tensorflow as tf
import numpy as np
import skimage.io
import sklearn.decomposition
import matplotlib.pyplot as plt
import pathlib
import pickle
import hyspec_cnn
import datetime

In [4]:
# Paths
base_dir = pathlib.Path('/massimal/data/Vega_Sola/Hyperspectral/20220823/Area')
train_tiles_path = base_dir / '3a_PCA_TrainValidationSplit/Training/PCA-Tiles/20220823_Vega_Sola_Train_Tiles'
val_tiles_path = base_dir / '3a_PCA_TrainValidationSplit/Validation/PCA-Tiles/20220823_Vega_Sola_Val_Tiles'

unet_model_save_dir = base_dir / 'M_UnetModels'
unet_model_save_dir.mkdir(exist_ok=True)
tensorboard_log_dir = base_dir / 'M_TensorBoardLogs'
tensorboard_log_dir.mkdir(exist_ok=True)

In [5]:
# Check if GPU is used
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Parameters
OUTPUT_CHANNELS = 8
BATCH_SIZE = 8
DEPTH = 2

In [7]:
# Load datasets 
train_dataset = tf.data.Dataset.load(str(train_tiles_path))
val_dataset = tf.data.Dataset.load(str(val_tiles_path))

In [8]:
# Get number of tiles in each dataset, and dataset shape
n_tiles_train = train_dataset.cardinality()
n_tiles_val = val_dataset.cardinality()
tile_nrows,tile_ncols,tile_nchannels = train_dataset.element_spec[0].shape.as_list()
print(f'Number of training tiles: {n_tiles_train}')
print(f'Number of validation tiles: {n_tiles_val}')
print(f'Tile data shape (PCA tiles): {(tile_nrows,tile_ncols,tile_nchannels)}')

Number of training tiles: 3266
Number of validation tiles: 711
Tile data shape (PCA tiles): (128, 128, 8)


In [9]:
# Weights
# {'Rock': 0.89,
#  'Cobble': 1.24,
#  'Sand': 0.82,
#  'Mearl bed': 0.65,
#  'Rockweed': 1.08,
#  'Kelp': 1.54,
#  'Brown algae': 0.77}

In [10]:
# From https://www.tensorflow.org/tutorials/images/segmentation#optional_imbalanced_classes_and_class_weights
def add_sample_weights(image, label, name):
    # class_weights = tf.constant([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) # Hard-coded for 7 classes
    class_weights = tf.constant([0.0, 0.89, 1.24, 0.82, 0.65, 1.08, 1.54, 0.77]) # Hard-coded for 7 classes
    class_weights = class_weights/tf.reduce_sum(class_weights)

    # Create an image of `sample_weights` by using the label at each pixel as an 
    # index into the `class weights` .
    sample_weights = tf.gather(class_weights, indices=tf.cast(label, tf.int32))

    return image, label, sample_weights

In [11]:
# Shuffle training dataset (tiles are originally ordered by image) and add sample weights
train_dataset = train_dataset.shuffle(buffer_size=n_tiles_train)
train_dataset = train_dataset.map(add_sample_weights)
val_dataset = val_dataset.map(add_sample_weights)

In [12]:
# Batch datasets
train_dataset_batch = train_dataset.batch(BATCH_SIZE)
val_dataset_batch = val_dataset.batch(BATCH_SIZE)

In [13]:
# Create the U-Net model
unet = hyspec_cnn.unet(input_channels=tile_nchannels,
                       output_channels=OUTPUT_CHANNELS,
                       first_layer_channels=32,
                       depth = DEPTH,
               )
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, None, None,  0           []                               
                                 8)]                                                              
                                                                                                  
 augmentation (Sequential)      (None, None, None,   0           ['input_image[0][0]']            
                                8)                                                                
                                                                                                  
 initial_convolution (Conv2D)   (None, None, None,   2336        ['augmentation[0][0]']           
                                32)                                                           

In [14]:
# Print layers with sublayers
for layer in unet.layers:
    print('----')
    print(layer.name)
    if hasattr(layer,'layers'):
        for l in layer.layers:
            print('\t'+l.name)
       # print(layer.layers)

----
input_image
----
augmentation
	random_flip
----
initial_convolution
----
downsamp_res_1/2
	conv2d
	batch_normalization
	leaky_re_lu
----
downsamp_res_1/4
	conv2d_1
	batch_normalization_1
	leaky_re_lu_1
----
upsamp_res_1/2
	conv2d_transpose
	batch_normalization_2
	re_lu
----
skipconnection_res_1/2
----
upsamp_res_1/1
	conv2d_transpose_1
	batch_normalization_3
	re_lu_1
----
skipconnection_res_1/1
----
classification


In [15]:
# Define callbacks
model_save_filename = str(unet_model_save_dir) + '/unet_model.depth' + str(DEPTH) +'.epoch{epoch:02d}-loss{val_loss:.6f}-acc{val_sparse_categorical_accuracy:.3f}.hdf5'
callbacks =[tf.keras.callbacks.ModelCheckpoint(filepath = model_save_filename,
                                               save_best_only=True,
                                               verbose = 1),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, verbose=1),
            tf.keras.callbacks.TensorBoard(log_dir= tensorboard_log_dir)]

In [16]:
model_save_filename

'/massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth2.epoch{epoch:02d}-loss{val_loss:.6f}-acc{val_sparse_categorical_accuracy:.3f}.hdf5'

In [17]:
# Compile model
unet.compile(optimizer=tf.keras.optimizers.RMSprop(0.0001), 
             loss="sparse_categorical_crossentropy",
             weighted_metrics=['sparse_categorical_accuracy'], # Need weights to ignore background
             metrics = []) # Sparse because classes are numbered, not one-hot

In [18]:
# print(tensorboard_log_dir)
#%tensorboard --logdir /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_TensorBoardLogs

In [19]:
# Fit model to dataset
history = unet.fit(train_dataset.batch(BATCH_SIZE),
                   epochs=100,
                   validation_data=val_dataset.batch(BATCH_SIZE),
                   callbacks=callbacks)

Epoch 1/100
409/409 [==============================] - ETA: 0s - loss: 0.0897 - sparse_categorical_accuracy: 0.5485
Epoch 1: val_loss improved from inf to 0.06365, saving model to /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth2.epoch01-loss0.063652-acc0.664.hdf5
409/409 [==============================] - 65s 118ms/step - loss: 0.0897 - sparse_categorical_accuracy: 0.5485 - val_loss: 0.0637 - val_sparse_categorical_accuracy: 0.6645 - lr: 1.0000e-04
Epoch 2/100
408/409 [============================>.] - ETA: 0s - loss: 0.0618 - sparse_categorical_accuracy: 0.6754
Epoch 2: val_loss improved from 0.06365 to 0.05732, saving model to /massimal/data/Vega_Sola/Hyperspectral/20220823/Area/M_UnetModels/unet_model.depth2.epoch02-loss0.057322-acc0.703.hdf5
409/409 [==============================] - 54s 112ms/step - loss: 0.0618 - sparse_categorical_accuracy: 0.6754 - val_loss: 0.0573 - val_sparse_categorical_accuracy: 0.7032 - lr: 1.0000e-04
Epoch 3/100
408/409 

Epoch 21/100
408/409 [============================>.] - ETA: 0s - loss: 0.0329 - sparse_categorical_accuracy: 0.8284
Epoch 21: val_loss did not improve from 0.03907
409/409 [==============================] - 46s 109ms/step - loss: 0.0329 - sparse_categorical_accuracy: 0.8284 - val_loss: 0.0408 - val_sparse_categorical_accuracy: 0.7959 - lr: 1.0000e-04
Epoch 22/100
408/409 [============================>.] - ETA: 0s - loss: 0.0317 - sparse_categorical_accuracy: 0.8345
Epoch 22: val_loss did not improve from 0.03907
409/409 [==============================] - 46s 110ms/step - loss: 0.0317 - sparse_categorical_accuracy: 0.8344 - val_loss: 0.0471 - val_sparse_categorical_accuracy: 0.7423 - lr: 1.0000e-04
Epoch 23/100
408/409 [============================>.] - ETA: 0s - loss: 0.0318 - sparse_categorical_accuracy: 0.8338
Epoch 23: val_loss did not improve from 0.03907
409/409 [==============================] - 46s 109ms/step - loss: 0.0318 - sparse_categorical_accuracy: 0.8338 - val_loss: 0.04

409/409 [==============================] - 47s 111ms/step - loss: 0.0266 - sparse_categorical_accuracy: 0.8621 - val_loss: 0.0412 - val_sparse_categorical_accuracy: 0.8048 - lr: 2.0000e-05
Epoch 42/100
408/409 [============================>.] - ETA: 0s - loss: 0.0254 - sparse_categorical_accuracy: 0.8680
Epoch 42: val_loss did not improve from 0.03620
409/409 [==============================] - 47s 111ms/step - loss: 0.0254 - sparse_categorical_accuracy: 0.8679 - val_loss: 0.0371 - val_sparse_categorical_accuracy: 0.8317 - lr: 4.0000e-06
Epoch 43/100
408/409 [============================>.] - ETA: 0s - loss: 0.0249 - sparse_categorical_accuracy: 0.8723
Epoch 43: val_loss did not improve from 0.03620
409/409 [==============================] - 47s 111ms/step - loss: 0.0249 - sparse_categorical_accuracy: 0.8723 - val_loss: 0.0372 - val_sparse_categorical_accuracy: 0.8346 - lr: 4.0000e-06
Epoch 44/100
408/409 [============================>.] - ETA: 0s - loss: 0.0254 - sparse_categorical_acc

KeyboardInterrupt: 